### **Import Package**

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

### **Import Data**

In [28]:
df = pd.read_csv('Online Retail Data.csv')
pd.set_option('display.max_column', None)
df.head()

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN


###**Eksplorasi Data**

In [29]:
# Cek struktur data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


In [30]:
# Cek statistika deskriptif
df.describe().round(2)

,quantity,price,customer_id
count,461773.00,461773.00,360853.00
mean,9.09,4.57,15557.61
std,86.37,153.41,1593.73
min,-9600.00,-53594.36,12346.00
25%,1.00,1.25,14210.00
50%,3.00,2.10,15580.00
75%,10.00,4.21,16938.00
max,10200.00,25111.09,18287.00


In [31]:
# Cek missing value
df.isnull().sum()

,0
order_id,0
product_code,0
product_name,2718
quantity,0
order_date,0
price,0
customer_id,100920


In [32]:
# Cek data duplicate
df.duplicated().sum()

np.int64(6479)

In [33]:
# Cek konsistensi nilai (duplikasi product_name untuk product_code yang sama)
desc_consistency = df.groupby('product_code')['product_name'].nunique().sort_values(ascending=False)
desc_consistency.head()

,product_name
product_code,
22423,6
22734,6
21523,4
21843,4
47566B,4


In [34]:
# Melihat duplikasi product_name untuk product_code "22423"
df.loc[df['product_code'] == '22423', 'product_name'].unique().tolist()

['REGENCY CAKESTAND 3 TIER',
 'smashed',
 'damaged',
 nan,
 'broken, uneven bottom',
 'wonky bottom/broken',
 'faulty']

### **Data Cleansing**

In [35]:
# Salin dataframe untuk pembersihan data
df_clean = df.copy()

In [36]:
# Menghapus semua baris tanpa customer_id
df_clean = df_clean[~df_clean['customer_id'].isna()]
# Menghapus semua baris tanpa product_name
df_clean = df_clean[~df_clean['product_name'].isna()]

In [37]:
# Mengubah customer_id ke format string
df_clean['customer_id'] = df['customer_id'].astype(str)
# Mengubah order_date ke format datetime
df_clean['order_date'] = pd.to_datetime(df_clean['order_date'])
# Membuat product_name berhuruf kecil
df_clean['product_name'] = df_clean['product_name'].str.lower()

In [38]:
# Menghapus nilai quantity negatif dan order_id yg diawali 'C' karena menandakan order "Cancelled"
df_clean.drop(df_clean[df_clean['quantity'] <= 0].index, inplace = True)
df_clean = df_clean[df_clean['order_id'].str[:1]!='C']
# Menghapus semua baris dengan price <= 0
df_clean.drop(df_clean[df_clean['price'] <= 0].index, inplace = True)
# Menghapus semua baris dengan product_code atau product_name test
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test'))]

In [39]:
# Mengganti product_name dari product_code yang memiliki beberapa product_name dengan salah satu product_name-nya yang paling sering muncul
most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')

In [40]:
# Membuat kolom amount, yaitu hasil kali antara quantity dan price
df_clean['amount'] = df_clean['quantity'] * df_clean['price']
# Membuat kolom year-month
df_clean['year_month'] = df_clean['order_date'].dt.to_period('M')

In [41]:
# Menghapus data duplicate
df_clean = df_clean.drop_duplicates()
df_clean.duplicated().sum()

np.int64(0)

In [42]:
# Menghapus outlier
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)

In [43]:
# product_code yang diawali dengan angka menunjukan kode produk, sementara
# product_code khusus seperti 'POST', 'D', dan 'M mewakili transaksi non-produk (jasa, penyesuaian, dll.)
# Analisis hanya berfokus pada transaksi produk (tanpa melibatkan data transaksi non produk)
df_clean = df_clean[df_clean['product_code'].str.match(r'^\d', na=False)]

df_clean.head()

,order_id,product_code,product_name,quantity,order_date,price,customer_id,amount,year_month
0,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,91.8,2010-01
1,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,51.0,2010-01
2,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,5.1,2010-01
3,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,8.5,2010-01
4,493414,21527,red retrospot traditional teapot,12,2010-01-04 10:28:00,6.95,14590.0,83.4,2010-01


In [44]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 343390 entries, 0 to 343818
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      343390 non-null  object        
 1   product_code  343390 non-null  object        
 2   product_name  343390 non-null  object        
 3   quantity      343390 non-null  int64         
 4   order_date    343390 non-null  datetime64[ns]
 5   price         343390 non-null  float64       
 6   customer_id   343390 non-null  object        
 7   amount        343390 non-null  float64       
 8   year_month    343390 non-null  period[M]     
dtypes: datetime64[ns](1), float64(2), int64(1), object(4), period[M](1)
memory usage: 26.2+ MB


###**RFM Segmentation**

**Agregat data transaksi ke bentuk summary:**
- total transaksi (order),
- total nilai order (order value), dan
- tanggal order terakhir dari setiap pengguna

In [45]:
df_clean.columns

Index(['order_id', 'product_code', 'product_name', 'quantity', 'order_date',
       'price', 'customer_id', 'amount', 'year_month'],
      dtype='object')

In [46]:
df_user = df_clean.groupby('customer_id', as_index=False).agg(order_cnt=('order_id','nunique'),max_order_date=('order_date','max'),total_order_value=('amount','sum'))
df_user

,customer_id,order_cnt,max_order_date,total_order_value
0,12346.0,2,2010-06-28 13:53:00,169.36
1,12608.0,1,2010-10-31 10:49:00,415.79
2,12745.0,2,2010-08-10 10:14:00,723.85
3,12746.0,1,2010-06-17 10:41:00,254.55
4,12747.0,14,2010-12-13 10:41:00,3723.94
...,...,...,...,...
3830,18283.0,6,2010-11-22 15:30:00,619.37
3831,18284.0,1,2010-10-04 11:33:00,411.68
3832,18285.0,1,2010-02-17 10:24:00,377.00
3833,18286.0,1,2010-08-20 11:57:00,833.48


**Membuat kolom jumlah hari sejak order terakhir**

In [47]:
today = df_clean['order_date'].max()
df_user['day_since_last_order'] = (today - df_user['max_order_date']).dt.days
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order
0,12346.0,2,2010-06-28 13:53:00,169.36,178
1,12608.0,1,2010-10-31 10:49:00,415.79,53
2,12745.0,2,2010-08-10 10:14:00,723.85,135
3,12746.0,1,2010-06-17 10:41:00,254.55,189
4,12747.0,14,2010-12-13 10:41:00,3723.94,10
...,...,...,...,...,...
3830,18283.0,6,2010-11-22 15:30:00,619.37,31
3831,18284.0,1,2010-10-04 11:33:00,411.68,80
3832,18285.0,1,2010-02-17 10:24:00,377.00,309
3833,18286.0,1,2010-08-20 11:57:00,833.48,125


In [48]:
df_user.describe().round(2)

,order_cnt,max_order_date,total_order_value,day_since_last_order
count,3835.00,3835,3835.00,3835.00
mean,4.23,2010-09-23 00:11:25.752281600,1484.33,91.42
min,1.00,2010-01-05 12:43:00,1.55,0.00
25%,1.00,2010-08-19 01:58:00,292.74,27.00
50%,2.00,2010-10-26 09:45:00,639.76,58.00
75%,5.00,2010-11-26 11:45:00,1548.76,126.00
max,144.00,2010-12-23 16:06:00,68627.68,352.00
std,6.87,NaN,3237.42,88.18


**Membuat binning dari jumlah hari sejak order terakhir yang terdiri dari 5 bins dengan batas-batasnya merupakan min, P20, P40, P60, P80, max dan beri label 1 sampai 5 dari bin tertinggi ke terendah sebagai skor recency**

In [49]:
df_user['recency_score'] = pd.cut(df_user['day_since_last_order'],
                                  bins=[df_user['day_since_last_order'].min(),
                                        np.percentile(df_user['day_since_last_order'], 20),
                                        np.percentile(df_user['day_since_last_order'], 40),
                                        np.percentile(df_user['day_since_last_order'], 60),
                                        np.percentile(df_user['day_since_last_order'], 80),
                                        df_user['day_since_last_order'].max()],
                                  labels=[5, 4, 3, 2, 1],
                                  include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score
0,12346.0,2,2010-06-28 13:53:00,169.36,178,1
1,12608.0,1,2010-10-31 10:49:00,415.79,53,3
2,12745.0,2,2010-08-10 10:14:00,723.85,135,2
3,12746.0,1,2010-06-17 10:41:00,254.55,189,1
4,12747.0,14,2010-12-13 10:41:00,3723.94,10,5
...,...,...,...,...,...,...
3830,18283.0,6,2010-11-22 15:30:00,619.37,31,4
3831,18284.0,1,2010-10-04 11:33:00,411.68,80,2
3832,18285.0,1,2010-02-17 10:24:00,377.00,309,1
3833,18286.0,1,2010-08-20 11:57:00,833.48,125,2


In [50]:
bins = [df_user['day_since_last_order'].min(),
        np.percentile(df_user['day_since_last_order'], 20),
        np.percentile(df_user['day_since_last_order'], 40),
        np.percentile(df_user['day_since_last_order'], 60),
        np.percentile(df_user['day_since_last_order'], 80),
        df_user['day_since_last_order'].max()]

# Menampilkan range untuk setiap Recency Score
for i in range(5):
    lower_bound = bins[i]
    upper_bound = bins[i+1]
    score = 5 - i
    print(f"Score {score}: {lower_bound:.1f} - {upper_bound:.1f} days")

Score 5: 0.0 - 22.0 days
Score 4: 22.0 - 42.0 days
Score 3: 42.0 - 77.0 days
Score 2: 77.0 - 165.0 days
Score 1: 165.0 - 352.0 days


In [51]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3835 entries, 0 to 3834
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3835 non-null   object        
 1   order_cnt             3835 non-null   int64         
 2   max_order_date        3835 non-null   datetime64[ns]
 3   total_order_value     3835 non-null   float64       
 4   day_since_last_order  3835 non-null   int64         
 5   recency_score         3835 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 179.9+ KB


**Membuat binning dari total transaksi (order) yang terdiri dari 5 bins dengan batas-batasnya merupakan min, P20, P40, P60, P80, max dan beri label 1 sampai 5 dari bin terendah ke tertinggi sebagai skor frequency**

In [52]:
df_user['frequency_score'] = pd.cut(df_user['order_cnt'],
                                    bins=[0,
                                          np.percentile(df_user['order_cnt'], 20),
                                          np.percentile(df_user['order_cnt'], 40),
                                          np.percentile(df_user['order_cnt'], 60),
                                          np.percentile(df_user['order_cnt'], 80),
                                          df_user['order_cnt'].max()],
                                    labels=[1, 2, 3, 4, 5],
                                    include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score
0,12346.0,2,2010-06-28 13:53:00,169.36,178,1,2
1,12608.0,1,2010-10-31 10:49:00,415.79,53,3,1
2,12745.0,2,2010-08-10 10:14:00,723.85,135,2,2
3,12746.0,1,2010-06-17 10:41:00,254.55,189,1,1
4,12747.0,14,2010-12-13 10:41:00,3723.94,10,5,5
...,...,...,...,...,...,...,...
3830,18283.0,6,2010-11-22 15:30:00,619.37,31,4,4
3831,18284.0,1,2010-10-04 11:33:00,411.68,80,2,1
3832,18285.0,1,2010-02-17 10:24:00,377.00,309,1,1
3833,18286.0,1,2010-08-20 11:57:00,833.48,125,2,1


In [53]:
bins = [0,
        np.percentile(df_user['order_cnt'], 20),
        np.percentile(df_user['order_cnt'], 40),
        np.percentile(df_user['order_cnt'], 60),
        np.percentile(df_user['order_cnt'], 80),
        df_user['order_cnt'].max()]

# Menampilkan range untuk setiap Frequency Score
print("Frequency Score Ranges:")
for i in range(5):
    lower_bound = bins[i]
    upper_bound = bins[i+1]
    score = i + 1
    print(f"Score {score}: {lower_bound:.1f} - {upper_bound:.1f} orders")

Frequency Score Ranges:
Score 1: 0.0 - 1.0 orders
Score 2: 1.0 - 2.0 orders
Score 3: 2.0 - 3.0 orders
Score 4: 3.0 - 6.0 orders
Score 5: 6.0 - 144.0 orders


In [54]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3835 entries, 0 to 3834
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3835 non-null   object        
 1   order_cnt             3835 non-null   int64         
 2   max_order_date        3835 non-null   datetime64[ns]
 3   total_order_value     3835 non-null   float64       
 4   day_since_last_order  3835 non-null   int64         
 5   recency_score         3835 non-null   int64         
 6   frequency_score       3835 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 209.9+ KB


**Membuat binning dari total nilai order (order value) yang terdiri dari 5 bins dengan batas-batasnya merupakan min, P20, P40, P60, P80, max dan beri label 1 sampai 5 dari bin terendah ke tertinggi sebagai skor monetary**

In [55]:
df_user['monetary_score'] = pd.cut(df_user['total_order_value'],
                                   bins=[df_user['total_order_value'].min(),
                                         np.percentile(df_user['total_order_value'], 20),
                                         np.percentile(df_user['total_order_value'], 40),
                                         np.percentile(df_user['total_order_value'], 60),
                                         np.percentile(df_user['total_order_value'], 80),
                                         df_user['total_order_value'].max()],
                                   labels=[1, 2, 3, 4, 5],
                                   include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score
0,12346.0,2,2010-06-28 13:53:00,169.36,178,1,2,1
1,12608.0,1,2010-10-31 10:49:00,415.79,53,3,1,2
2,12745.0,2,2010-08-10 10:14:00,723.85,135,2,2,3
3,12746.0,1,2010-06-17 10:41:00,254.55,189,1,1,2
4,12747.0,14,2010-12-13 10:41:00,3723.94,10,5,5,5
...,...,...,...,...,...,...,...,...
3830,18283.0,6,2010-11-22 15:30:00,619.37,31,4,4,3
3831,18284.0,1,2010-10-04 11:33:00,411.68,80,2,1,2
3832,18285.0,1,2010-02-17 10:24:00,377.00,309,1,1,2
3833,18286.0,1,2010-08-20 11:57:00,833.48,125,2,1,3


In [56]:
bins = [df_user['total_order_value'].min(),
        np.percentile(df_user['total_order_value'], 20),
        np.percentile(df_user['total_order_value'], 40),
        np.percentile(df_user['total_order_value'], 60),
        np.percentile(df_user['total_order_value'], 80),
        df_user['total_order_value'].max()]

# Menampilkan range untuk setiap Monetary Score
print("Monetary Score Ranges:")
for i in range(5):
    lower_bound = bins[i]
    upper_bound = bins[i+1]
    score = i + 1
    print(f"Score {score}: $ {lower_bound:,.0f} - $ {upper_bound:,.0f}")

Monetary Score Ranges:
Score 1: $ 2 - $ 234
Score 2: $ 234 - $ 464
Score 3: $ 464 - $ 898
Score 4: $ 898 - $ 1,934
Score 5: $ 1,934 - $ 68,628


In [57]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3835 entries, 0 to 3834
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3835 non-null   object        
 1   order_cnt             3835 non-null   int64         
 2   max_order_date        3835 non-null   datetime64[ns]
 3   total_order_value     3835 non-null   float64       
 4   day_since_last_order  3835 non-null   int64         
 5   recency_score         3835 non-null   int64         
 6   frequency_score       3835 non-null   int64         
 7   monetary_score        3835 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(1)
memory usage: 239.8+ KB


**Membuat kolom nama segmen berdasarkan skor recency dan frequency**

In [58]:
df_user['segment'] = np.select(
    [
        (df_user['recency_score'] == 5) & (df_user['frequency_score'] >= 4),
        (df_user['recency_score'].between(3, 4)) & (df_user['frequency_score'] >= 4),
        (df_user['recency_score'] >= 4) & (df_user['frequency_score'].between(2, 3)),
        (df_user['recency_score'] <= 2) & (df_user['frequency_score'] == 5),
        (df_user['recency_score'] == 3) & (df_user['frequency_score'] == 3),
        (df_user['recency_score'] == 5) & (df_user['frequency_score'] == 1),
        (df_user['recency_score'] == 4) & (df_user['frequency_score'] == 1),
        (df_user['recency_score'] <= 2) & (df_user['frequency_score'].between(3, 4)),
        (df_user['recency_score'] == 3) & (df_user['frequency_score'] <= 2),
        (df_user['recency_score'] <= 2) & (df_user['frequency_score'] <= 2)
    ],
    [
        '01-Champion',
        '02-Loyal Customers',
        '03-Potential Loyalists',
        "04-Can't Lose Them",
        '05-Need Attention',
        '06-New Customers',
        '07-Promising',
        '08-At Risk',
        '09-About to Sleep',
        '10-Hibernating'
    ],
    default='11-Unknown'
)

**Summary RFM segmentation, berupa banyaknya pengguna, rata-rata dan median dari total order, total order value, dan jumlah hari sejak order terakhir**

In [59]:
summary = pd.pivot_table(df_user, index='segment',
               values=['customer_id','day_since_last_order','order_cnt','total_order_value'],
               aggfunc={'customer_id': 'nunique',
                        'day_since_last_order': ['mean', 'median'],
                        'order_cnt': ['mean', 'median'],
                        'total_order_value': ['mean', 'median']})
summary['pct_unique'] = (summary[('customer_id', 'nunique')] / summary[('customer_id', 'nunique')].sum() * 100).round(1)
summary.round(2)

customer_id day_since_last_order        order_cnt  \
                           nunique                 mean median      mean   
segment                                                                    
01-Champion                    570                11.63   11.0     12.15   
02-Loyal Customers             591                42.43   38.0      6.73   
03-Potential Loyalists         395                23.94   25.0      2.43   
04-Can't Lose Them              46               126.89  117.0      9.93   
05-Need Attention              115                60.03   60.0      3.00   
06-New Customers                65                14.95   17.0      1.00   
07-Promising                   162                33.04   34.0      1.00   
08-At Risk                     329               136.24  116.0      3.81   
09-About to Sleep              424                59.27   59.0      1.42   
10-Hibernating                1138               193.69  193.0      1.29   

                              total_order_value          pct_unique  
                       median              mean   median             
segment                                                              
01-Champion               8.0           4530.31  2537.00       14.9  
02-Loyal Customers        6.0           2429.68  1762.99       15.4  
03-Potential Loyalists    2.0            729.29   595.26       10.3  
04-Can't Lose Them        8.0           3116.03  2114.81        1.2  
05-Need Attention         3.0           1077.08   885.79        3.0  
06-New Customers          1.0            259.69   207.50        1.7  
07-Promising              1.0            298.97   248.48        4.2  
08-At Risk                4.0           1269.42  1027.03        8.6  
09-About to Sleep         1.0            496.72   359.26       11.1  
10-Hibernating            1.0            373.79   279.75       29.7

In [60]:
# Simpan dataframe user
df_user.to_csv('Dataframe RFM.csv', index=False, float_format="%.3f")